# OPENCV NOTEBOOK

#  Thresholding
Thresholding is the binarization of an image. In general,
we seek to convert a grayscale image to a binary image,
where the pixels are either 0 or 255.

A simple thresholding example would be selecting a pixel
value p, and then setting all pixel intensities less than p to
zero, and all pixel values greater than p to 255. In this way,
we are able to create a binary representation of the image.

Normally, we use thresholding to focus on objects or areas of particular interest in an image. In the examples in the
sections below, we will empty our pockets and look at our
spare change. Using thresholding methods, we’ll be able to
find the coins in an image.

In [1]:
import cv2
import mahotas
import numpy as np

In [2]:
img = cv2.imread("images\symbol.jpg")
img.shape

(588, 466, 3)

## Simple Thresholding
Applying simple thresholding methods requires human intervention. We must specify a threshold value T. All pixel
intensities below T are set to 0. And all pixel intensities
greater than T are set to 255.

We can also apply the inverse of this binarization by setting all pixels below T to 255 and all pixel intensities greater
than T to 0.

In [3]:
# First convert the BGR image to Gray scale image
image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("Original Image", image)
blurred = cv2.GaussianBlur(image, (5, 5), 0)
cv2.imshow("Gaussian Blurred Image", blurred)
cv2.waitKey(0)

-1

In [4]:
# Threshold Binary
(T, thres) = cv2.threshold(blurred, 140, 255, cv2.THRESH_BINARY)
cv2.imshow("Threshold Binary Image", thres)

(T, invThres) = cv2.threshold(blurred, 140, 255, cv2.THRESH_BINARY_INV)
cv2.imshow("Inverse Threshold Binary Image", invThres)
cv2.waitKey(0)

-1

In [5]:
# Now, apply the invThres image as mask to the image
cv2.imshow("Image", cv2.bitwise_and(image, image, mask=invThres))
cv2.waitKey(0)

-1

## Adaptive Thresholding

In [6]:
meanThres = cv2.adaptiveThreshold(blurred, # Gray Scale Image
                              255,     # Max Pixels in the image
                              cv2.ADAPTIVE_THRESH_MEAN_C,  # Mean for neighborHood pixels
                              cv2.THRESH_BINARY_INV,    # Threshold binary inverse
                              11, # Neighborhood size
                              4)   # C (C - mean)
cv2.imshow("Mean Thresh", meanThres)
gausThres = cv2.adaptiveThreshold(blurred, # Gray Scale Image
                              255,     # Max Pixels in the image
                              cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  # Mean for neighborHood pixels
                              cv2.THRESH_BINARY_INV,    # Threshold binary inverse
                              15, # Neighborhood size
                              3)   # C (C - gaussian value)
cv2.imshow("Gaussian Thres", gausThres)
cv2.waitKey(0)

-1

We then apply adaptive thresholding to our blurred image using the cv2.adaptiveThreshold function The first parameter we supply is the image we want to threshold. Then, we supply our maximum value of 255, similar to simple thresholding mentioned above.

The third argument is our method to compute the threshold for the current neighborhood of pixels. By supplying cv2.ADAPTIVE_THRESH_MEAN_C, we indicate that we want to compute the mean of the neighborhood of pixels and treatit as our T value.

Next, we need our thresholding method. Again, the description of this parameter is identical to the simple thresholding method mentioned above. We use cv2.THRESH_BINARY_INV to indicate that any pixel intensity greater than T inthe neighborhood should be set to 255, otherwise it should
be set to 0.

The next parameter is our neighborhood size. This integer value must be odd and indicates how large our neighborhood of pixels is going to be. We supply a value of 11, indicating that we are going to examine 11 × 11 pixel regions of the image, instead of trying to threshold the image globally, as in simple thresholding methods.

Finally, we supply a parameter simply called C. This value is an integer that is subtracted from the mean, allowing us to fine-tune our thresholding. We use C = 4 in this example. The results of applying mean weighted adaptive thresholding can be seen in the output.

Besides applying standard mean thresholding, we can also apply Gaussian (weighted mean) thresholding. The order of the parameters are identical to adaptive threshold mean , but now we are tuning a few of the values. Instead of supplying a value of cv2.ADAPTIVE_THRESH_ MEAN_C, we instead use cv2.ADAPTIVE_THRESH_GAUSSIAN_C to indicate we want to use the weighted mean.

We are also using a 15 × 15 pixel neighborhood size rather than an 11 × 11 neighborhood size as in the previous example. We also alter our C value (the value we subtract from the mean) slightly and use 3 rather than 4. The results of applying Gaussian adaptive thresholding and mean adaptive thresholding has a little difference between the two images.

In general, choosing between mean adaptive thresholding and Gaussian adaptive thresholding requires a few experiments on your end. The most important parameters to vary are the neighborhood size and C, the value you subtract from the mean. By experimenting with this value, you will be able to dramatically change the results of your thresholding.

## Otsu and Riddler-Calvard
Another way we can automatically compute the threshold
value of T is to use Otsu’s method.

Otsu’s method assumes there are two peaks in the grayscale
histogram of the image. It then tries to find an optimal
value to separate these two peaks – thus our value of T.
While OpenCV provides support for Otsu’s method, I
prefer the implementation by Luis Pedro Coelho in the mahotas
package since it is more Pythonic.

In [3]:
# First convert our BGR to gray scale image and then apply the gaussian Blur method
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("Blurred Original Image", blurred)
cv2.waitKey(0)

-1

In [5]:
# Otsu's Threshold
cv2.imshow("Original", img)
# Maunally create the threshold using otsu and rc (Riddle-Calvard)
T = mahotas.thresholding.otsu(blurred)
print(f"Otsu Threshold: {T}")

# Create the copy of the image for thresholding
thres = gray.copy()
thres[thres > T] = 255
thres[thres < T] = 0
thresh = cv2.bitwise_not(thres)
cv2.imshow("Otsu", thresh)
cv2.waitKey(0)

Otsu Threshold: 148


-1

In [6]:
# Riddle Calvard's Threshold
cv2.imshow("Original", img)
# Maunally create the threshold using otsu and rc (Riddle-Calvard)
T = mahotas.thresholding.rc(blurred)
print(f"Riddle Calvard Threshold: {T}")

# Create the copy of the image for thresholding
thres = gray.copy()
thres[thres > T] = 255
thres[thres < T] = 0
thresh = cv2.bitwise_not(thres)
cv2.imshow("Riddle Calvard", thresh)
cv2.waitKey(0)

Riddle Calvard Threshold: 148.67064178674718


-1